## Property Mining Project 

This project is designed to parse property websites in Dubai. The parser initially goes onto the Dubizzle's site, gathers data on all the listed apartments, their essential traits and areas and then stores it in a local database. The database is an intermediary for better data processing and avoiding data errors. 

The next series of the project is the data visualization of the data gathered w.r.t to the area, amenities and the phase 2 of the project will also include integration with google maps API to see the nearest amenities and based on the distance from nearest amenities an OLS regression will be run to see the variability on price based on the data features. 

In [ ]:
import re
import mysql.connector
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import datetime
import requests
import random
import numpy as np
import pandas as pd
from time import sleep
import codecs

### Fetching Apartments

In [2]:
mydb = mysql.connector.connect(host = 'localhost', user='root', password='****') ## Setting up connection with local database
cur = mydb.cursor(buffered=True)

In [3]:
cur.execute('use dubbizle')

In [32]:
wish = input('Enter the type you would like to fetch:') # Entery point for villa or apartment 
fetch_type = str(wish)

try:
    if fetch_type == 'apartment':
        extract_url = 'https://dubai.dubizzle.com/en/property-for-rent/residential/apartmentflat/?page='
        apartmentType = 'apartment'
    elif fetch_type == 'villa':
        extract_url = 'https://dubai.dubizzle.com/en/property-for-rent/residential/villahouse/?page='
        apartmentType = 'villa'
    else: 
        print('Nothing to Fetch')
except:
    sys.exit()


table = dict()
area = dict()
refId= []
num = [1,2,3,4]

for x in range(0,40):
    print('\n Number of Pages: {}'.format(x))        
    try:
        http = requests.get(extract_url+str(x))
        bs = BeautifulSoup(http.text, 'html.parser')
        main_page = bs.select('div.main-body a')
    except requests.exceptions.RequestException:
        print('There is an Error')
        continue
    for x in main_page:
        if not x.attrs['href'].startswith('https://'):
            try:
                http = requests.get('https://dubai.dubizzle.com'+x.attrs['href'])
                bs1 = BeautifulSoup(http.text, 'html.parser')
            except requests.exceptions.RequestException:
                print('Encountered an Error')
                continue
            try:    
                desc = bs1.find('h1')
                demo = bs1.find_all(property = 'name') #list to be made
                refNo = bs1.find('em')
                price = bs1.find(text= re.compile('^(AED).*')) #it is text
                bed = bs1.find_all('p',{'class':'Text__Root-sc-1q498l3-0 Text___StyledRoot-sc-1q498l3-1 jQBcNQ CardList___StyledText-hqnz3a-1 zbsnO'}) 
                rentType = bs1.find('div',{'data-ui':'Rent Is Paid-value'})
                desctype = bs1.find_all('p',{'class':'Text__Root-sc-1q498l3-0 Text___StyledRoot-sc-1q498l3-1 bzZrcx'})
                datePosted = bs1.find('div',{'data-ui':'Posted On-value'})
            except AttributeError:
                print('There is Page is missing something')
                continue
            
            for (x,y) in zip(bed,desctype):
                table[y.text] = x.text

            for x,y in zip(demo,num):
                area['area_{}'.format(y)] = x.text
            
            if rentType is None:
                rentType = None
            else:
                rentType = rentType.text

            if refNo is None:
                refNo = None
            else:
                refNo = refNo.text
            
            
            if datePosted is None:
                datePosted = None
            else:
                datePosted = datePosted.text

            if desc is None:
                desc = None
            else:
                desc = desc.text

            
            print(desc, refNo, str(price),apartmentType, rentType, datePosted)

            
            try:
                cur.execute('select * from location where description = %s',(desc,) )
                if cur.fetchone() is None:
                    

                    try:
                        cur.execute('''insert into location (description,referral,price, type, rent,postDate) 
                                values (%s,%s,%s,%s,%s,%s)''',
                                    (desc,refNo,str(price),apartmentType,rentType, datePosted))

                        cur.execute('''insert into details (beds, baths, size, furnish, area_1, area_2, area_3, area_4,
                        area_5)values (%s,%s,%s,%s,%s,%s,%s,%s,%s)''',
                                    (table.get('Bedrooms', None),table.get('Bathrooms', None),
                                                                table.get('Size (SqFt)',None),table.get('Furnished',None),
                                                                area.get('area_1',None),area.get('area_2',None),
                                                                area.get('area_3',None),area.get('area_4',None), 
                                                                area.get('area_5',None)))


                    except:
                        print('There is a missing Value')
                        continue
                        
                else:
                    pass
            except:
                print("Last Error")
                continue
        
            table.clear()
            area.clear()
            mydb.commit()    
    
    

Enter the type you would like to fetch: apartment



 Number of Pages: 0
Free Month | New Bright 2BR | 3 Bathroom | Laundry Room | High Quality Building and Amenities | Cove TOTH-10XX01-TFO AED 85,000/year in 4 Cheques apartment Quarterly September 16th, 2020
Brand New | Chiller Free | Kitchen Appliances | Excellent View | 12 cheques AZCO-JVC-AL-JW-2BR-7825 AED 60,000 /yr apartment Monthly September 23rd, 2020
PAY 4CHQS | PRIME LOCTION @24K FLO2-JVC1-929 AED 24,000 /yr apartment Quarterly September 23rd, 2020
BRAND NEW SPACIOUS 1BR WITH PARKING ONLY 30k AL WARQA SNK SHAKTHI  282 AED 30,000 /yr apartment Quarterly September 23rd, 2020
Spacious 1 BR Opposite Sharjah American School 1377-Ap-R-0407 AED 38,000 /yr apartment Quarterly September 23rd, 2020
FULL FURNISHED SMALL 1 BHK TYPE STUDIO  ONLY 30K /-4CH TO 35K 4CH SJD-DG1-4CH AED 30,000 /yr apartment Yearly September 19th, 2020
Fully Furnished | Ready to Move AZCO-UM-SIRAJ TOWER-1BR-2498-1 AED 43,999 /yr apartment Yearly September 23rd, 2020
Great Offer | 1 BHK | Near Lulu Village A1735

In [431]:
mydb.commit()